<a href="https://colab.research.google.com/github/thingumajig/colab-experiments/blob/master/predict_BERT_BiLSTM_CRF_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

## clone from git

In [1]:
#!pip install --upgrade tensorflow-gpu
#!pip install seqeval
!pip install bert-tensorflow

!pip list | grep tensorflow
#!ps ax | grep python


     |████████████████████████████████| 71kB 4.3MB/s 
bert-tensorflow          1.0.1                
mesh-tensorflow          0.0.5                
tensorflow               1.14.0               
tensorflow-estimator     1.14.0               
tensorflow-hub           0.5.0                
tensorflow-metadata      0.14.0               
tensorflow-probability   0.7.0                


In [2]:

import sys

# bert_path = 'bert'
# !test -d $bert_path || git clone https://github.com/google-research/bert $bert_path
# if not bert_path in sys.path:
#   sys.path.insert(0, bert_path)



bert_ner_path = 'bert-ner'  
!test -d $bert_ner_path || git clone https://github.com/thingumajig/colab-experiments $bert_ner_path

if not bert_ner_path in sys.path:
  sys.path.insert(0, bert_ner_path)

Cloning into 'bert-ner'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 79 (delta 39), reused 24 (delta 8), pack-reused 0
Unpacking objects: 100% (79/79), done.


In [45]:
#!rm -rfv $bert_ner_path/bert
!git -C $bert_ner_path pull
if not bert_ner_path in sys.path:
  sys.path.insert(0, bert_ner_path)

Already up to date.


##Model

In [10]:
#@title Select model:
import os

bert_model = "https://storage.googleapis.com/bert_models/2019_05_30/wwm_cased_L-24_H-1024_A-16.zip" #@param ["https://storage.googleapis.com/bert_models/2019_05_30/wwm_cased_L-24_H-1024_A-16.zip", "https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip"]
bert_model_zip = os.path.basename(bert_model)
bert_model_dir = os.path.splitext(bert_model_zip)[0]

!test -d $bert_model_dir || wget $bert_model


!test -d $bert_model_dir || unzip /content/$bert_model_zip

--2019-07-26 19:08:44--  https://storage.googleapis.com/bert_models/2019_05_30/wwm_cased_L-24_H-1024_A-16.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.128, 2607:f8b0:400e:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1242589256 (1.2G) [application/zip]
Saving to: ‘wwm_cased_L-24_H-1024_A-16.zip’

wwm_cased_L-24_H-10 100%[===================>]   1.16G   114MB/s    in 9.2s    

2019-07-26 19:08:54 (128 MB/s) - ‘wwm_cased_L-24_H-1024_A-16.zip’ saved [1242589256/1242589256]

Archive:  /content/wwm_cased_L-24_H-1024_A-16.zip
   creating: wwm_cased_L-24_H-1024_A-16/
  inflating: wwm_cased_L-24_H-1024_A-16/bert_model.ckpt.meta  
  inflating: wwm_cased_L-24_H-1024_A-16/bert_model.ckpt.data-00000-of-00001  
  inflating: wwm_cased_L-24_H-1024_A-16/vocab.txt  
  inflating: wwm_cased_L-24_H-1024_A-16/bert_model.ckpt.index  
  inflating: wwm_cased_L-24_H-10

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


##init flags

In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import os
import json

import tensorflow as tf
import codecs

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

flags = tf.flags
FLAGS = flags.FLAGS

# print(FLAGS.__dict__['__wrapped'])
def del_all_flags(FLAGS, r = []):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        if keys in r:
          FLAGS.__delattr__(keys)
 
# allows rerun cell!
del_all_flags(FLAGS, r = ['bert_config_file','data_dir',
                         "task_name", "output_dir","init_checkpoint",
                         "do_lower_case", "do_train", "use_tpu", "do_predict",
                         "use_crf", "train_batch_size", "eval_batch_size",
                         "learning_rate", "num_train_epochs", "warmup_proportion",
                         "bert_dropout_rate", "bilstm_dropout_rate",
                          "save_checkpoints_steps", "save_summary_steps",
                         'data_config_path', "num_tpu_cores", "master",
                          "vocab_file", "keep_checkpoint_max",                          
                          'lstm_size', 'max_seq_length',
                          'predict_batch_size', 'iterations_per_loop',
                          'f',
                          
                         ])

train_model_path = '/content/drive/My Drive/ner_checkpoints'
if not os.path.exists(train_model_path):
    os.makedirs(train_model_path)

flags.DEFINE_string("data_dir", f'/content/{bert_ner_path}/data/CoNLL', "The input datadir. ex) 'NERdata'")
flags.DEFINE_string("bert_config_file", os.path.join('/content',bert_model_dir,'bert_config.json'), "The config json file corresponding to the pre-trained BERT model. ex) 'bert_config.json'")

flags.DEFINE_string("task_name", 'NER', "The name of the task to train. ex) 'NER'")

flags.DEFINE_string("output_dir", train_model_path, "The output directory where the model checkpoints will be written. ex) 'output'")

flags.DEFINE_string("init_checkpoint", f"/content/{bert_model_dir}/bert_model.ckpt", "Initial checkpoint (usually from a pre-trained BERT model).")

flags.DEFINE_bool("do_lower_case", False, "Whether to lower case the input text.")

flags.DEFINE_integer("max_seq_length", 128, "The maximum total input sequence length after WordPiece tokenization.")

flags.DEFINE_bool("do_train", False, "Whether to run training.")
flags.DEFINE_bool("do_predict", True, "Whether to run the model in inference mode on the test set.")

flags.DEFINE_bool("use_crf", True, "Whether to use CRF decoding.")

flags.DEFINE_integer("train_batch_size", 16, "Total batch size for training.")
flags.DEFINE_integer("eval_batch_size", 8, "Total batch size for eval.")
flags.DEFINE_integer("predict_batch_size", 8, "Total batch size for predict.")
flags.DEFINE_float("learning_rate", 5e-5, "The initial learning rate for Adam.")

flags.DEFINE_float("num_train_epochs", 3.0, "Total number of training epochs to perform.")
flags.DEFINE_float("warmup_proportion", 0.1, "Proportion of training to perform linear learning rate warmup for. E.g., 0.1 = 10% of training.")
flags.DEFINE_float("bert_dropout_rate", 0.2, "Proportion of dropout for bert embedding.")
flags.DEFINE_float("bilstm_dropout_rate", 0.2, "Proportion of dropout for bilstm.")
flags.DEFINE_integer("save_checkpoints_steps", 1000, "How often to save the model checkpoint.")
flags.DEFINE_integer("save_summary_steps", 100, "Save summaries every this many steps")
flags.DEFINE_integer("keep_checkpoint_max", 20, "The maximum number of recent checkpoint files to keep")
flags.DEFINE_integer("iterations_per_loop", 1000, "How many steps to make in each estimator call.")

flags.DEFINE_string("vocab_file", f'/content/{bert_model_dir}/vocab.txt', "The vocabulary file that the BERT model was trained on. ex) 'vocab.txt'")

flags.DEFINE_string("master", None, "[Optional] TensorFlow master URL.")

flags.DEFINE_bool("use_tpu", False, "Whether to use TPU or GPU/CPU.")
flags.DEFINE_integer( "num_tpu_cores", 8, "Only used if `use_tpu` is True. Total number of TPU cores to use.")

flags.DEFINE_string('data_config_path', 'data.conf', 'data config file, which save train and dev config')

flags.DEFINE_integer('lstm_size', 128, 'size of lstm units')

flags.DEFINE_string("f", 'f', "f") #????? error otherwise

print(FLAGS.__dict__['__wrapped'])



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:
  --bert_config_file: The config json file corresponding to the pre-trained BERT
    model. ex) 'bert_config.json'
    (default: '/content/wwm_cased_L-24_H-1024_A-16/bert_config.json')
  --bert_dropout_rate: Proportion of dropout for bert embedding.
    (default: '0.2')
    (a number)
  --bilstm_dropout_rate: Proportion of dropout for bilstm.
    (default: '0.2')
    (a number)
  --data_config_path: data config file, which save train and dev config
    (default: 'data.conf')
  --data_dir: The input datadir. ex) 'NERdata'
    (default: '/content/bert-ner/data/CoNLL')
  --[no]do_lower_case: Whether to lower case the input text.
    (default: 'false')
  --[no]do_predict: Whether to run the model in inference mode on the test set.
    (default: 'true')
  --[no]do_train: Whether to run training.
    (default: 'false')
  --eval_batch_size: Total batch size for eval.
    (default: '8')
    (an integer)
  --init_checkpoint: Initial

# run

In [12]:
import bert_bilstm_model as m
from estimator_train_predict import *

processors = {
    "ner": m.NerProcessor
}

task_name = FLAGS.task_name.lower()
if task_name not in processors:
    raise ValueError("Task not found: %s" % (task_name))
processor = processors[task_name]()
label_list = processor.get_labels()


estimator, tokenizer = create_estimator(label_list)


W0726 19:09:36.467666 139861836302208 estimator.py:1984] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f33c02f0ae8>) includes params argument, but params are not passed to Estimator.
W0726 19:09:36.469970 139861836302208 tpu_context.py:211] eval_on_tpu ignored because use_tpu is False.


================== use WarmStartSettings from /content/drive/My Drive/ner_checkpoints


In [14]:
text = '''
The Puerto Rican Legislature gave Gov. Ricardo Rosselló a choice by saying that they're ready to start impeachment proceedings unless he resigns first.

The president of Puerto Rico’s House of Representatives, Carlos Méndez Núñez, has called for an extraordinary legislative session to officially begin an impeachment process against Rosselló Thursday afternoon.
The news came on Wednesday after three attorneys, commissioned by Méndez Núñez, unanimously determined there were five offenses, stemming from leaked scandalous chats, that constitute grounds for impeachment.

In the report that was leaked to the press, the attorneys found Rosselló committed four serious offenses and one misdemeanor, including illicitly using public resources and services for partisan purposes, as well as allowing government officials and contractors to misuse public funds and time for non-government work.

During a press conference, Méndez Núñez said the only thing that would stop the process is if the governor resigns.

If Rosselló steps down, then the report would be passed on to the relevant authorities, he added.

On Wednesday evening, a crowd of thousands had gathered in Old San Juan near the governor's mansion, shouting "Ricky, Renuncia!"

One of the protesters, Adrián, told NBC News that "tonight was supposed to be a dance party." But since Rosselló has consistently refused to resign, he's afraid things will get ugly.

“This will be bad,” said Adrián, who brought his gas mask and goggles as a precaution measure.

On Wednesday evening, a large number of reporters were assembled for hours at the door of the governor's mansion, waiting for what was expected to be a much earlier announcement from Rosselló or his aides about his future.

Rosselló's public affairs secretary, Anthony Maceira, addressed reporters two hours later, but only said that the governor was working on a statement that he would give at an unspecified time.

The turmoil follows the island's largest protest in recent history calling for Rosselló's ouster over scandals involving leaked private chats as well as corruption investigations and arrests.

News of Rosselló's impeachment process came a day after NBC News and Telemundo, both owned by NBC Universal, reported that the island's Justice Department had issued search warrants to confiscate the cellphones of several people who took part in the private chats.

Hundreds of thousands of Puerto Ricans have been protesting for 12 consecutive days, demanding Rosselló's ouster. Protests continued to grow on the island after Rosselló announced on Sunday that he wouldn't run for re-election and that he would step down from the leadership of the pro-statehood New Progressive Party.

On Monday, more than a half-million people paralyzed metropolitan San Juan in protest, marching across one of the main highways despite heavy rain in a "March of the People" that ended late in the night as police fired tear gas canisters.
'''

text1 = '''
Republicans in the Senate have twice in 24 hours blocked the advancement of bills aimed at strengthening election security just hours after former special counsel Robert Mueller warned of the continued threat that foreign powers interfering in US elections.

Senate Majority Leader Mitch McConnell came to the Senate floor Thursday to personally object to House-passed legislation backed by Democrats. This comes after Republican Sen. Cindy Hyde-Smith of Mississippi objected to a trio of bills on Wednesday, in keeping with long standing GOP arguments that Congress has already responded to election security needs for the upcoming election.
Democratic Sens. Mark Warner of Virginia, Richard Blumenthal of Connecticut and Ron Wyden of Oregon had advocated for the bills on the Senate floor, asking for unanimous consent to pass the package, but that ask can be halted with an objection from any senator.
Two of those bills would require campaigns to report to federal authorities any attempts by foreign entities to interfere in US elections, and the third is aimed at protecting from hackers the personal accounts and devices of senators and some staffers.
On Thursday, McConnell called the House-passed legislation Democrats wanted to clear "so partisan it received just one Republican vote over in the House." He added that any election security legislation the Senate takes up must be bipartisan and also alleged that this bill is being pushed by the same Democrats who pushed the "conspiracy theory" of President Donald Trump and Russia.
'''

import pickle
with codecs.open(os.path.join(FLAGS.output_dir, 'label2id.pkl'), 'rb') as rf:
  label2id = pickle.load(rf)
  id2label = {value: key for key, value in label2id.items()}

for (id, lbl) in id2label.items():
  print(f'{id}={lbl}')

predict_examples, result = predict(processor, estimator, tokenizer, label_list, text)
result = list(result)



W0726 19:10:36.120021 139861836302208 deprecation_wrapper.py:119] From bert-ner/bert_bilstm_model.py:298: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.



1=O
2=B-PER
3=I-PER
4=B-ORG
5=I-ORG
6=B-LOC
7=I-LOC
8=B-MISC
9=I-MISC
10=X
================= Number of predict examples:527
Writing example 0 of 527
*** Example ***
guid: predict-0
tokens: The
input_ids: 101 1109 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
input_mask: 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

W0726 19:10:36.934286 139861836302208 deprecation.py:323] From bert-ner/bert_bilstm_model.py:338: map_and_batch (from tensorflow.contrib.data.python.ops.batching) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
W0726 19:10:36.935658 139861836302208 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/data/python/ops/batching.py:273: map_and_batch (from tensorflow.python.data.experimental.ops.batching) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.data optimizations will take care of using the fused implementation.
W0726 19:10:36.938788 139861836302208 deprecation_wrapper.py:119] From bert-ner/bert_bilstm_model.py:318: The name tf.parse_single_example is deprecated. Please use tf.io.parse_single_example instead.



input_fn: batch_size=8 is_training=False
*** Features ***
  name = input_ids, shape = (?, 128)
  name = input_mask, shape = (?, 128)
  name = label_ids, shape = (?, 128)
  name = segment_ids, shape = (?, 128)
shape of input_ids (?, 128)
shape of label_ids (?, 128)
estimator mode: infer


W0726 19:10:42.220675 139861836302208 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


seq_length 128
lengths Tensor("Sum:0", shape=(?,), dtype=int32)


W0726 19:10:43.191802 139861836302208 deprecation.py:323] From bert-ner/bert_bilstm_model.py:384: dropout (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dropout instead.
W0726 19:10:43.440197 139861836302208 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/crf/python/ops/crf.py:213: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W0726 19:10:43.535346 139861836302208 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_ops.py:2403: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0726 19:10:43.774132 1398618

####################
shape of output_layer: (?, 128, 1024)
embedding_size:1024
seq_length:128
shape of logit (?, 128, 11)
shape of loss ()
shape of per_example_loss (?,)
num labels:11
####################
shape of pred_ids (?, 128)


W0726 19:10:45.909057 139861836302208 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [50]:
from render_results import *


#    output_predict_file = os.path.join(FLAGS.output_dir, "pred.txt")
#     write_predictions(predict_examples, result, output_predict_file, id2label)
from IPython.display import HTML, display
display(HTML(
  render_predictions_html_v2(predict_examples, result, id2label)
))


In [16]:
from render_results import render_predictions
render_predictions(predict_examples, result, id2label)

The Puerto<B-LOC> Rican<B-MISC> Legislature gave Gov .
Ricardo<B-PER> Rosselló<B-PER> a choice by saying that they 're ready to start impeachment proceedings unless he resigns first .
The president of Puerto<B-LOC> Rico<B-LOC> ’ s House of Representatives , Carlos<B-PER> Méndez<B-PER> Núñez<B-PER> , has called for an extraordinary legislative session to officially begin an impeachment process against Rosselló<B-PER> Thursday afternoon .
The news came on Wednesday after three attorneys , commissioned by Méndez<B-PER> Núñez<B-PER> , unanimously determined there were five offenses , stemming from leaked scandalous chats , that constitute grounds for impeachment .
In the report that was leaked to the press , the attorneys found Rosselló<B-PER> committed four serious offenses and one misdemeanor , including illicitly using public resources and services for partisan purposes , as well as allowing government officials and contractors to misuse public funds and time for non-government work .
D

In [38]:
for (id, lbl) in id2label.items():
  print(f'{id}={lbl}')

for pe, r in zip(predict_examples, result):
  length = len(pe.tokenized_tokens)
  tgs = list(map(lambda x: id2label[x], r[1:length + 1]))
  print(f'{str(pe.tokens):<18}|{str(pe.tokenized_tokens):<35}|{str(pe.tokenized_labels):<20}|{str(tgs):<20}')
print(predict_examples[6].tokens)
print(result[6])

1=O
2=B-PER
3=I-PER
4=B-ORG
5=I-ORG
6=B-LOC
7=I-LOC
8=B-MISC
9=I-MISC
10=X
['The']           |['The']                            |['O']               |['O']               
['Puerto']        |['Puerto']                         |['O']               |['B-LOC']           
['Rican']         |['Rican']                          |['O']               |['B-MISC']          
['Legislature']   |['Legislature']                    |['O']               |['O']               
['gave']          |['gave']                           |['O']               |['O']               
['Gov']           |['Go', '##v']                      |['O', 'X']          |['O', 'X']          
['.']             |['.']                              |['O']               |['O']               
['Ricardo']       |['Ricardo']                        |['O']               |['B-PER']           
['Rosselló']      |['Ross', '##ell', '##ó']           |['O', 'X', 'X']     |['B-PER', 'X', 'X'] 
['a']             |['a']                            

In [0]:
#print(FLAGS.__dict__['__wrapped'])
flags.DEFINE_string("f", 'f', "f")
FLAGS.data_dir

'/content/bert-ner/CoNLL/data'

In [0]:
!ps ax | grep python

import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize, sent_tokenize
#my_sent = "Hi man, how have you been?"
#print(word_tokenize(my_sent))

ls = sent_tokenize(text)
for (i,l) in enumerate(ls): 
  print(f'====== line: {i} ============')
  print(l)



#!zip model.ckpt-2632.zip "/content/drive/My Drive/ner_checkpoints/model.ckpt-2000.data-00000-of-00001" "/content/drive/My Drive/ner_checkpoints/model.ckpt-2632.index" "/content/drive/My Drive/ner_checkpoints/model.ckpt-2632.meta" "/content/drive/My Drive/ner_checkpoints/label2id.pkl"

     28 ?        Sl     0:03 /usr/bin/python2 /usr/local/bin/jupyter-notebook --ip="172.28.0.2" --port=9000 --FileContentsManager.root_dir="/" --MappingKernelManager.root_dir="/content"
    133 ?        Ssl    0:07 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-317cbf9d-4c91-465b-bf56-800c930e6f59.json
    422 ?        S      0:00 /usr/bin/python3 -Wignore:::pip._internal.cli.base_command -c from multiprocessing.semaphore_tracker import main;main(58)
    546 ?        S      0:00 /bin/bash -c ps ax | grep python
    548 ?        S      0:00 grep python
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
====== line: 0 ============

INTERLOCAL AGREEMENT FOR CONSULTING SERVICES

This is an Interlocal Agreement entered into pursuant to the authority of ch.
====== line: 1 ============
39.34 RCW, between the Cities of Lacey, Olympia, Tumwater, and Thurston County, hereinafter referred to as the JU

In [0]:
from IPython.display import HTML, display
xxx = """
<style> 
.x1 {
	  padding: .2em .3em;
    padding-top: 0.2em;
    padding-right: 0.3em;
    padding-bottom: 0.2em;
    padding-left: 0.3em;
    margin: 0 .25em;
    margin-top: 0px;
    margin-right: 0.25em;
    margin-bottom: 0px;
    margin-left: 0.25em;
    line-height: 1;
    display: inline-block;
    border-radius: .25em;
    border-top-left-radius: 0.25em;
    border-top-right-radius: 0.25em;
    border-bottom-right-radius: 0.25em;
    border-bottom-left-radius: 0.25em;
}

.x2 {
    box-sizing: border-box;
    content: attr(data-entity);
    font-size: .55em;
    line-height: 1;
    padding: .35em .35em;
    padding-top: 0.35em;
    padding-right: 0.35em;
    padding-bottom: 0.35em;
    padding-left: 0.35em;
    border-radius: .35em;
    text-transform: uppercase;
    display: inline-block;
    vertical-align: middle;
    margin: 0 0 .15rem .5rem;
    margin-top: 0px;
    margin-right: 0px;
    margin-bottom: 0.15rem;
    margin-left: 0.5rem;
    background: #fff;
    background-image: initial;
    background-position-x: initial;
    background-position-y: initial;
    background-size: initial;
    background-repeat-x: initial;
    background-repeat-y: initial;
    background-attachment: initial;
    background-origin: initial;
    background-clip: initial;
    background-color: rgb(255, 255, 255);
    font-weight: 700;
}


</style>

<p>Rounded corners for an element with a specified background color:</p>kljhkhlkj
<mark class="x1" style='background-color: red;'>Rounded corners!<sub class="x2">PERS</sub></mark>
<p>Rounded corners for an element with a border:</p>
<p id="rcorners2">Rounded corners!</p>
<p>Rounded corners for an element with a background image:</p>

"""



display(HTML(xxx), metadata=dict(isolated=True))